# Chapter 9
In this chapter we look at:
* Classes and class relationships
* Syntax/code-idioms
* Exhaustive search
* Previous Chapters

In [chapter 1](chapter1.ipynb) (a closely related notebook see hyperlink) We look at optimization outputs that came from an optimization using spike waveform shape. 

In [chapter 2](chapter2.ipynb) (a closely related notebook see hyperlink) I demonstrate optimization using spike time statistics via the allen SDK Chapter 2

In [chapter 3](chapter3.ipynb) (a closely related notebook see hyperlink) We will take a closer at the neuroelectro data used to perform the fits in notebook1/Chapter 1. Specifically we will look at where optimized model behavior fits back onto the distribution of neuroelectro data.

In [chapter 5](chapter5.ipynb) (a closely related notebook see hyperlink) We will look at projections of Optimized cells onto a Druckman feature space, we will also look at extracting Allen SDK features from the optimized cells. Chapter 3


# Syntax and Idioms
## Data Transport Container
The main way of storing model attributes and moving non-parallel models tests around on different CPUs

Class Definition is in /neuronunit/neuronunit/optimisation/data_transport_container.py

Inputs and outputs of Optimization, are typically Data Transport Containers. 

dtc objects now contain a method, in order to quickly recover the model represented by a particular data transport container.
``` python
model = dtc.dtc_to_model()
```
DataTransport Container also includes a method judge tests, to judge any test suites that may be stored inside a model.

``` python
model = dtc.judge_tests()
```
..., and a method to convert dtc objects to genes:


``` python
model = dtc.dtc_to_gene()
```

## Test Suite Dictionary
An extension of pythons dictionary class. It is capable of calling a method ```optimize``` on itself.

* This type also has an attribute, use_rheobase in scores.

* Since not all experimental cell test suites, use rheobase to judge model fitness (Allen SDK does not). It's simpler to decide if rheobase calculations will contribute to Neuronunit scores at the users entrypoint to optimization.

 /neuronunit/neuronunit/optimisation/optimization_management.py

## Class OptMan
* An Optimization Management Class

Located at
 /neuronunit/neuronunit/optimisation/optimization_management.py

This object contains the method ```run_ga```, ```round_trip_test``` and ```run_grid```.
The object contains logic for switching between elephant and allen based measurements of waveform and firing. This object also contains logic for rheobase guided searches, or many firing waveform searches.

Once an optimization has been described using the Optmization management object a secondary object is constructed.

The ```OptMan``` object also performs tasks like creating new genes to replace models that had unstable parameters

# Some examples of using the OptMan class to optimize:


``` python
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})

grid_results = OM.run_simple_grid(5,free_params=['a','b','c'])
```

To simulate data, and internall check multi-objective high dimensional optimization:

``` python

OM = OptMan(use_test,protocol={'elephant':True,'allen':False,'dm':False})
results,converged,target,simulated_tests = OM.round_trip_test(use_test,
                                                              str('RAW'),\
                                                              MU=10,NGEN=10,stds = easy_standards)

```
To optimize against external neuro-electro data with an axaptive exponential backend:
``` python
ga_outad, DO = OM.run_ga(model_params.MODEL_PARAMS['ADEXP'],3, local_tests, free_params = model_params.MODEL_PARAMS['ADEXP'], MU = 10, backend = str('ADEXP'))
```
The run_ga method is now wrapped inside a ***Neuronunit Test Suite*** dictionary class. 

``` python
# test_dic an actual simple dictionary of NU Tests:
keys: test names, values test objects:

sim_tests = TSD(test_dic)

dtcpop = sim_tests.optimize(model_parameters.MODEL_PARAMS[backend], NGEN=50, \
                             backend=backend, MU=24, protocol={'allen': True, 'elephant': False,'tsr':results['dtc_pop'][0].tsr})

```



This Object constructs another object called  ```SciUnitOptimization``` that takes care of deap configuration. This file is departs from BluePyOpts base class, but it is similar to it.

The class sciunitoptimization largely calls methods in the files:
* neuronunit/optimisation/algorithms.py***
* neuronunit/optimisation/optimisations.py***

* Two files still largely follow BluePyOpt and DEAP conventions (expressed in DEAPS nsga2.py example file):

* ***algorithms.py***
* ***optimisations.py***

* With the differences: dask.bag.map is used to parallel map instead of scoop etc.

* nsga3 is used in the place of nsga2

* algorithms -> optimization_management.py ```def evaluate()``` 


This class (appropriated from BluePyOpt) is used to create new genes to replace models with unstable behavior.
```python
class WSListIndividual(list):
    """Individual consisting of list with weighted sum field"""
    def __init__(self, *args, **kwargs):
        """Constructor"""
        self.rheobase = None
        super(WSListIndividual, self).__init__(*args, **kwargs)

```



In a very simple example below. The OptMan class is instanced and used to perform simple optimization jobs including:
Exhaustive Search, running a genetic algorithm (nsga3), or round trip testing with simulated data


In [1]:
%%capture
from allensdk.ephys.extract_cell_features import extract_cell_features
import pickle
from neuronunit.optimisation.optimization_management import OptMan, TSD
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
import pandas as pd
import os



In [2]:
import os
electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)
local_tests = test_frame['Hippocampus CA1 pyramidal cell']
local_tests = TSD(local_tests)
local_tests.use_rheobase_score = True    


In [3]:
%%capture
free_params=['b','C','peak_v']
import copy
for f in free_params:
    hold_constant = copy.copy(MODEL_PARAMS['ADEXP'])
    hold_constant.pop(f,'None')

OM = OptMan(local_tests,backend=str('ADEXP'),\
            boundary_dict=MODEL_PARAMS['ADEXP'],\
            protocol={'elephant':True,'allen':False,'dm':False},\
            hc=hold_constant)#'tsr':spk_range})

ga_results = OM.optimize(free_params=['b','C','peak_v'],MU=5,NGEN=5)


In [23]:
# res = OM.round_trip_test(local_tests,str('RAW'),MU=1,NGEN=1)#,stds = easy_standards)
ga_results['pf'][0].dtc.scores
ga_results['pf'][0].dtc.scores_ratio
print(ga_results['dtc_pop'][0].scores)
local_tests

{'RheobaseTest': 0.9999999999998739}


{'CapacitanceTest': <neuronunit.tests.passive.CapacitanceTest at 0x7f3ba3d26400>,
 'InjectedCurrentAPAmplitudeTest': <neuronunit.tests.waveform.InjectedCurrentAPAmplitudeTest at 0x7f3ba3d263c8>,
 'InjectedCurrentAPThresholdTest': <neuronunit.tests.waveform.InjectedCurrentAPThresholdTest at 0x7f3ba3d26358>,
 'InjectedCurrentAPWidthTest': <neuronunit.tests.waveform.InjectedCurrentAPWidthTest at 0x7f3ba3d26438>,
 'InputResistanceTest': <neuronunit.tests.passive.InputResistanceTest at 0x7f3ba3d26630>,
 'RestingPotentialTest': <neuronunit.tests.passive.RestingPotentialTest at 0x7f3ba3d26320>,
 'RheobaseTest': <neuronunit.tests.fi.RheobaseTest at 0x7f3ba3d25518>,
 'TimeConstantTest': <neuronunit.tests.passive.TimeConstantTest at 0x7f3ba3d26390>}

In [27]:
%%capture
# restrict the number of free parameters to 3, to make the grid search computationally tractable.
#MODEL_PARAMS['ADEXP']
grid_results,sorted_pop = OM.run_simple_grid(2,free_params=['b','C','peak_v'])

In [28]:
opt=ga_results['pf'][0].dtc.scores_ratio
opt

In [29]:
if type(opt) is not type(None):
    print('its {0}, that ga beat grid'.format(opt<grid_results[0].dtc.scores))
    grid_results[-1].dtc.scores
    len(grid_results)
    print(grid_results[-1].dtc.scores_ratio)
    print(grid_results[0].dtc.scores_ratio)


In [30]:

%%capture
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})
# restrict the number of free parameters to 3, to make the grid search computationally tractable.
grid_results,sp = OM.run_simple_grid(5,free_params=['a','b','C'])


In [25]:
%%capture

ga_results_izh = OM.optimize(free_params=['a','b','C'],MU=3,NGEN=3)

len([ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ])

True
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b767404a8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b763c62e8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76740e10>]
OrderedDict([('vt', -45.0), ('c', -50.0), ('vPeak', 37.5), ('vr', -65.5), ('k', 1.15), ('d', 80.0), ('dt', 0.005)])
OrderedDict([('vt', -45.0), ('c', -50.0), ('vPeak', 37.5), ('vr', -65.5), ('k', 1.15), ('d', 80.0), ('dt', 0.005)])
OrderedDict([('vt', -45.0), ('c', -50.0), ('vPeak', 37.5), ('vr', -65.5), ('k', 1.15), ('d', 80.0), ('dt', 0.005)])
3
[[50, 0.01, -2], [50, 0.03, 15], [200, 0.03, 15]]
[[50, 0.010270846096002161, -2], [50, 0.02972915390399784, 15], [200, 0.03, 15]]
[[50, 0.010270846096002161, -2], [50, 0.02972915390399784, 15], [200, 0.03, 15]]
True
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b7678a

125

In [11]:
[ (gr.dtc.scores,gr.dtc.attrs,gr) for gr in grid_results ] 

[({'CapacitanceTest': 1.0,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'InjectedCurrentAPThresholdTest': 0.9797489593517121,
   'InjectedCurrentAPWidthTest': 0.5781685020940115,
   'InputResistanceTest': 1.0,
   'RestingPotentialTest': 0.36816528919597813,
   'RheobaseTest': 8.444470342672261e-05,
   'TimeConstantTest': 1.0},
  {'C': 50.0, 'a': 0.01, 'b': -2.0},
  [50.0, -2.0, 0.01]),
 ({'CapacitanceTest': 1.0,
   'InjectedCurrentAPAmplitudeTest': 0.9980351630935769,
   'InjectedCurrentAPThresholdTest': 0.9797386958247887,
   'InjectedCurrentAPWidthTest': 0.5465633794237801,
   'InputResistanceTest': 1.0,
   'RestingPotentialTest': 0.36816528919597813,
   'RheobaseTest': 0.00012896870066358979,
   'TimeConstantTest': 1.0},
  {'C': 50.0, 'a': 0.01, 'b': 2.25},
  [50.0, 2.25, 0.01]),
 ({'CapacitanceTest': 1.0,
   'InjectedCurrentAPAmplitudeTest': 0.9980349614128396,
   'InjectedCurrentAPThresholdTest': 0.9797579247216706,
   'InjectedCurrentAPWidthTest': 0.608443302246897

# Sort this list to find optimal solution from grid search.
* Use optimal solution to seed a genetic algorithm.

This will be a fast way of showing that the genetic algorithm can find a better solution. Alternatively we could start the GA from scratch, but if the genetic algorithm was not working at all, this would be a faster way to demonstrate it. Since time is short, let us do it the fast way first.



In [12]:
dtc_pop = sorted([ (gr.dtc.scores_ratio,gr.dtc.attrs,gr) for gr in grid_results ], key=lambda tup: tup[0])

# Ground truth via exhaustive search is:


In [13]:
print('The model implied by parameters: {0}'.format(dtc_pop[0][1]))
print('Yields the scores: {0}'.format(dtc_pop[0][2].dtc.scores))
print('Yields the ratio scores: {0}'.format(dtc_pop[0][2].dtc.scores_ratio))

The model implied by parameters: {'C': 200.0, 'b': -2.0, 'a': 0.019999999999999997}
Yields the scores: {'InjectedCurrentAPThresholdTest': 0.9797592588416886, 'TimeConstantTest': 0.6897795369461786, 'RestingPotentialTest': 0.36816528919597813, 'InjectedCurrentAPWidthTest': 0.0, 'CapacitanceTest': 1.0, 'InputResistanceTest': 1.0, 'RheobaseTest': 8.467818562785112e-05, 'InjectedCurrentAPAmplitudeTest': 0.9980353270758306}
Yields the ratio scores: 0.629478011280663


* Now lets create a seed population from the top 10 exhaustive search results.


In [14]:
gene_pop = [d[2] for d in dtc_pop[0:10]]
len(dtc_pop)

125

In [15]:
%%capture 
results = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop), free_params=dtc_pop[0][1].keys(), \
                               seed_pop=gene_pop)
#(NGEN = 10, MU=len(gene_pop), seed_pop = gene_pop,backend='RAW',)
#[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

# Optimization management classes: 
* can also do things like round-trip testing (test the simulators internal performance, as if to bench mark itself with simulated neural data).

In [22]:
results[0]['dtc_pop'][0].scores
opt_out = results[0]['dtc_pop'][0].scores_ratio
opt_out

0.6295326599763822

In [17]:
grid_out = dtc_pop[0][2].dtc.scores_ratio

print('its true that ga beat exhaustive search {0}'.format(opt_out<grid_out))
print('its true that ga was no worse than exhaustive search {0}'.format(opt_out==grid_out))

print(results[0]['dtc_pop'][0].attrs,'grid best parameters')
print(dtc_pop[0][2].dtc.attrs,'grid best parameters')
grid_out

its true that ga beat exhaustive search False
its true that ga was no worse than exhaustive search False
{'C': 200.0, 'b': -0.40872513244019437, 'a': 0.03} grid best parameters
{'C': 200.0, 'b': -2.0, 'a': 0.019999999999999997} grid best parameters


0.629478011280663

In [18]:
grid_out = dtc_pop[0][2].dtc.scores_ratio
opt_out

0.6295326599763822

In [19]:
results_no_seed = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop),\
                               free_params=dtc_pop[0][1].keys())


True
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e4a8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e6d8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e1d0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e2b0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e550>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e518>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3e748>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a3edd8>]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
{'amplitude': array(0.07854793) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude':

{'amplitude': array(0.07913168) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07913168) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.11391232) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.11391232) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.1338905) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11391232) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.1338905) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11391232) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.1338905) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.1338905) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
{'amplitude': 

True
{'amplitude': array(0.11406672) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11406672) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
{'amplitude': array(0.08384085) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
{'amplitude': array(0.08384085) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
{'amplitude': array(0.13318387) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13318387) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13318387) * pA, 'delay': array(1

True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
True
{'amplitude': array(0.07823207) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) *

True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
True
True
True
[[200, 0.03, -2], [50, 0.01, 15], [193.35174677115594, 0.02829170007350354, 14.60925910279

...

True
{'amplitude': array(0.13986817) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14409334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13986817) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14409334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.13986817) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13986817) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
{'amplitude': array(0.07950927) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
Z = 0.48
{'amplitude': array(0.07950927) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578

{'amplitude': array(0.13953358) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
{'amplitude': array(0.13953358) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13953358) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
Z = 0.48
{'v

True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06

{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
Z = 0.48
Z = 0.48
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522

Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
{'amplitude': array(0.08025041) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.08025041) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.08025041) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.08025041) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
{'

In [20]:
grid_results_4 = OM.run_simple_grid(4,free_params=['a','b','c','C'])
results_no_seed_4 = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop),\
                               free_params=dtc_pop[0][1].keys())



['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a81cc0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a81940>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a81a90>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b769e8048>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a99e10>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a81128>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b769e86a0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76a813c8>]
True
True
True
True
True
True
{'amplitude': array(0.0785633) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.0785633) * p

True
True
True
True
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10843531) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10843531) * 

True
True
True
True
True
{'amplitude': array(0.07832854) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07832854) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07832854) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07832854) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
{'amplitude': array(0.10406641) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10406641) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.1040664

True
True
True
True
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
True
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.12155006) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.12155006) * 

True
True
True
True
True
{'amplitude': array(0.10629872) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10629872) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10629872) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10629872) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
['c', 'C', 'b', 'a']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b7678f898>

...

Z = 0.48
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
Z = 0.48
True
True
True
True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': a

Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
[[200, 0.03, -0.4430100516387685], [199.71197674344327, 0.03, -1.97692601087146], [53.92971953753381, 0.029934951313782154, 15], [198.19793260928068, 0.029823446462098772, -0.9509961680748953], [51.41617915959526, 0.010000882290269773, 14.783491102012865], [200, 0.010532701804954274, 

['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
['C', 'a', 'b']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b762a6278>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b762a67b8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b762a6a20>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76ae12b0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76ae1898>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76ae1278>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x7f3b76ae1c18>]
True
True
True
True
True
True
True
True
True
True
True
{'amplitude': array(0.14064814) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14064814) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.14064814) * pA, 'delay': 

{'amplitude': array(0.17790571) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.08964459) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.17790571) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
{'amplitude': array(0.07822271) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
Z = 0.48
True
True
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
{'amplitude': array(0.13213652) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
{'amplitude': array(0.13318799) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std':

Z = 0.48
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
True
Z = 0.48
{'value': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV, 'mean': array(-79.) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
True
The model implied by parameters: {'C': 200.0, 'b': -2.0, 'a': 0.019999999999999997}
Yields the scores: {'InjectedCurrentAPThresholdTest': 0.9797592588416886, 'TimeConstantTest': 0.6897795369461786, 'RestingPotentialTest': 0.36816528919597813, 'InjectedCurrentAPWidthTest': 0.0, 'CapacitanceTest': 1.0, 'InputResistanceTest': 1.0, 'RheobaseTest': 8.467818562785112e-05, 'InjectedCurrentAPAmplitudeTest': 0.9980353270758306}
Yields the ratio scores: 0.629478011280663


In [21]:
%%capture


UsageError: %%capture is a cell magic, but the cell body is empty.


In [26]:
results_no_seed_4

({'dtc_pop': [<neuronunit.optimisation.data_transport_container.DataTC at 0x7f3b76a81320>,
  'gen_vs_pop': [[[50, 0.01, -2],
    [50, 0.01, 15],
    [50, 0.03, -2],
    [50, 0.03, 15],
    [200, 0.01, -2],
    [200, 0.01, 15],
    [200, 0.03, -2],
    [200, 0.03, 15]],
   [[50, 0.01, -1.1178073022726602],
    [50, 0.01, 15],
    [50, 0.03, -2],
    [50, 0.02978788158060123, 14.511809035124255],
    [200.0, 0.01, -0.4833110019963356],
    [200, 0.01, 13.528557059603038],
    [200.0, 0.03, 14.740869123616996],
    [200, 0.03, -1.4176650697611812]],
   [[50, 0.029787996543543405, 14.511809035124255],
    [200, 0.02999644521578999, -2],
    [197.24732332960576, 0.010422833873255115, 15],
    [189.98370169993913, 0.029856233614713127, 15],
    [200.0, 0.028693576826237535, -1.9956445473455662],
    [50, 0.011306423173762466, -0.4279749871364865],
    [200, 0.03, -1.4176650697611812],
    [50, 0.01, -2]],
   [[50, 0.028554998665128367, 14.511809035124255],
    [200.0, 0.010002596564728804, -

In [36]:
print('Yields the scores: {0}'.format(results_no_seed_4[0]['pf'][0].dtc.scores))
print('Yields the ratio scores: {0}'.format(results_no_seed_4[0]['pf'][0].dtc.scores_ratio))
print('The model implied by parameters: {0}'.format(grid_results_4[0][1].dtc.scores_ratio))
print('it seems like the optimizer can often beat the grid search but it really depends')

Yields the scores: {'InjectedCurrentAPThresholdTest': 0.97975628204651, 'TimeConstantTest': 0.6897672243576218, 'InjectedCurrentAPWidthTest': 0.04270619187466207, 'CapacitanceTest': 1.0, 'RestingPotentialTest': 0.36816528919597813, 'InputResistanceTest': 1.0, 'RheobaseTest': 8.441105308176766e-05, 'InjectedCurrentAPAmplitudeTest': 0.9980353270758306}
Yields the ratio scores: 0.6348143407004605
The model implied by parameters: 0.7405253153026199
